# Intro to Flask: Lesson 4

### Databases

So far, we've just learned how to grab user information with forms on Flask, but we don't know how to store that information long term.


In this lesson, we will store learn how to store user-inputted information by linking our Flask Applications to a database. 

We will do this with SQL, structured query language, which will allow you to store data in a format similar to a large Excel sheet:

![SegmentLocal](https://www.dummies.com/wp-content/uploads/414095.image0.jpg "segment")

In a lot of cases, it would be beneficial to learn SQL syntax and statements to work with SQL databases, but we can use libraries that allow us to just use Python code, so that we don't have to learn a whole new language.

We will connect our python and flask to SQLite, which is a simple SQL database engine that can handle everything we need.

To connect all three though, we need an ORM, object rotational mapper, which can directly use python instead of SQL syntax to do anything we might need, such as create or updating from our database.

The most common ORM for python is SQLAlchemy, but we will use Flask-SQLAlchemy, which is a further extension that allows for the connection of Flask with SQLAlchemy. To install:

In [ ]:
pip install Flask-SQLAlchemy

To start working with databases, we must do the following things:

1) Set up an SQLite Database in a Flask application

2) Create a model in Flask application

3) Create, Read, Update, and Delete data in the model. These basic database operations are also known as CRUD.

### Setting up SQLite database in a Flask application

You start this by creating a Flask app, like we have before:

In [ ]:
app = Flask(__name__)

Then you need to configure flask app for SQLAlchemy, in a similar way to configuring a secret key when working with forms:

In [ ]:
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data,sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

Then you will pass app into a SQLAlchemy class call:

In [ ]:
db = SQLAlchemy(app)

And you're free to make a model and work with it in Python. We will see this in an example later in the lesson, so that you will get a better understanding of it.

### Create a model in Flask App

![SegmentLocal](https://media0.giphy.com/media/evYq0C369LiQ8/giphy.gif "segment")

Models are important because they directly link to a table in an SQL database, so you do not have to manually create the table, you just have to create a model class that makes the table for us!

The process is similar to creating a FlaskForm, except for models:

1) You create a model class

2) Inherit from db.model

3) optionally provide a table name

4) Add in table columns as attributes

5) and add methods for __init__ and __rep__


Again, this will make more sense when you see it in an example.

## Example

![SegmentLocal](https://media0.giphy.com/media/3o6ZtaO9BZHcOjmErm/giphy.gif "segment")

The example below explains the concepts above with correct syntax. Here, create_database.py creates a table and manually adds two puppies (written in code). Since models were used we did not have to do this (users would do this on a website, etc), but it's useful to see as an initial example:

In [ ]:
## basic.py
import os
## allows us to grab directory names and filepath names programmatically with python
from flask import Flask 
from flask_sqlalchemy import SQLAlchemy

basedir = os.path.abspath(os.path.dirname(__file__))
##__file__ is built in and set to name of file, aka basic.py, os.path grabs the name and then the file path
## this gives you something like C://Desktop/flaskwork/basic.py

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'data,sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
## These create a SQLite database using python 

class Puppy(db.Model):
    ## creating the table model 
    __tablename__ = 'puppies'
    
    id = db.Column(db.Integer, primary_key = True)
    ## id is an attribute in the class, and is set equal to a column of integers. 
    ## making it a primary key means that the ID attribute will be unique for each puppy, so it should be the primary 
    ## distinguisher, unlike names or breeds. If you want to look for a specific puppy, you would use its id.  
    name = db.Column(db.Text)
    age = db.Column(db.Integer)
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def __rep__(self):
        return "puppy {self.name} is {self.age} years old"

In [ ]:
# create_database.py
from basic import db, Puppy
## takes from basic.py

db.create_all() 
## creates all tables, models become db tables

sam = Puppy('sammy', 3)
# This is basically saying: sam is an instance of Puppy class, named sammy, 3 years old 
frank = Puppy('frankie', 4)

print(sam.id)
print(frank.id)
# These shpuld both print None because they are not yet in the database

db.session.add_all([sam, frank])
# You can also just add one by using .add(sam)

db.session.commit()
# Saves changes

print(sam.id)
print(frank.id)

Notice that when you run create_database.py, it will print:

None

None

1

2

because indexing for databases starts at 1 rather than 0.

This is the first time you have been introduced to apps that link multiple files in order to work. Note that you only import Flask and SQLAlchemy in basic.py. However, createdatabase.py still uses those packages effortlessly. This is because createdatabase.py imports basic.py and all the packages it uses - it's dependent on the main file. Thus here and in future lessons, basic.py or app.py will be the main file, and all supplementary files imports that code - and perhaps other files' code - according to its dependencies. If done correctly, the code files link and can run together.   

The next part is just to show you the basics of CRUD - Creating, Reading, Updating, and Deleting:

In [ ]:
# crud.py
from basic import db, puppy

# create 
my_puppy = Puppy('Rufus', 5)
db.session.add(my_puppy)
db.session.commit()
# same we saw in last one 

# Read
all_puppies = Puppy.query.all()
# returns a list of puppy objects in table
print(all_puppies)

# select by ID:
puppy_one = Puppy.query.get(1)
print(puppy_one.name)

# filters:
puppy_frankie = Puppy.query.filter_by(name = 'Frankie')
print(puppy_frankie.all())
# prints all puppies with name frankie
# prints in the form: puppy Frankie is 3 years old 

# Update
first_puppy = Puppy.query.get(1)
first_puppy.age = 10
db.session.add(first_puppy)
db.session.commit()

# Delete
second_pup = Puppy.query.get(2)
db.session.delete(second_pup)
db.session.commit()

# Source Jose Portilla

To see your results, run crud.py. Play around with this for a while. What happens when you print Puppy.query.get(2)? How would print sammy's age? 

### Flask Migrate

![SegmentLocal](https://media3.giphy.com/media/U1keBkXnIeiY0/source.gif "segment")

When you make a Model for a database table, there will be times where you need to make changes to the Model - if you want to add a new column to a data table, then revert it back to the previous state, for example. 

Database migration refers to the management of these incremental, reversible changes and version control of the database. When you make any change, you need to "migrate" the changes so the table itself gets updated, much like you do in git. Then you'll have a history of changes that you can revert or look at any time.

We will use Flask Migrate to do this!

To install:

In [ ]:
pip install Flask-Migrate

Using this allows us to link changes we make to our Model class with our SQL database.

To use Migrate in your app, with the imports of your basic.py code, include:

In [ ]:
from flask_migrate import Migrate 

and directly beneath db = SQLAlchemy(app), include:

In [ ]:
Migrate(app, db) 

This connects the application with the database.

Migration involves four main commands for the command line.

First you need to set the FLASK_APP environment variable, to tell Flask how to load your document:

In [ ]:
## MacOS/Linux:
export FLASK_APP=basic.py

## Windows:
set FLASK_APP=basic.py

## basic.py is whatever the name of your flask application is

Remember, we refer to the file's directory when we do this. So when running the above, you need to make sure you are in the same directory as your file!!

Next, you can add a migrations directory to your application:

In [ ]:
flask db init

If you type "ls" into the command line, you can see that the migrations directory has been created, along with a pycache directory. The pycache directory holds optimized versions of your files and can be ignored. The migrations directory can also be ignored; just note that that is where you will be putting your migrated database files.

Next set up the migrations file:

In [ ]:
flask db migrate -m "some message"    # For this app, "created puppy table" could be useful.

When you do this and further steps, you might receive info tags, such as:

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.

INFO  [alembic.runtime.migration] Will assume non-transactional DDL.

INFO  [alembic.env] Detected added table 'puppies'

This gives you information on the save and can be ignored for now.

Note - the message in the migrate command is used to communicate the context of a save (what was updated or removed, why changes were made, etc) to others and your future self. If you look at your history of changes, the message will indicate what you were changing at that moment. 




To update the database with the migration:

In [ ]:
flask db upgrade

To summarize, you can run the four commands we just went over on your command line:

In [ ]:
export FLASK_APP=basic.py
flask db init
flask db migrate -m "some message"
flask db upgrade

Go ahead and do this before continuing.

We can now modify our database - add a column, for example. 

To do this, within the Puppy class of our code, we will add a breed attribute:


In [ ]:
breed = db.Column(db.Text)

This specifies a new text column for our database.

You should also assign a value for breed:

In [ ]:
self.breed = breed 

Finally, we need to save these changes since we added a column and rearranged the whole database:

In [ ]:
flask db migrate -m "added breed column"
flask db upgrade

This column is empty at the moment. If anybody wants to add their dog's breed in the future, or you want to input another dog into the system, you can put their breed alongside their name and age. 

Next, on the command line, run:

In [ ]:
flask db history

Here you will see that there is a record of your migrations with your commit messages. You can go back and see your old version or revert the change whenever you want.

# Homework

Begin using databases to store your users data/for what your app needs them for.